In [1]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu116.html


In [2]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv


In [3]:
edge_idx = torch.tensor([
                         [0, 1], 
                         [1, 0], 
                         [1, 2],
                         [2, 1]
                        ], 
                        dtype=torch.long)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_idx.t().contiguous())
print(data)
print(f'data.x = 3 nodes each with 1 feature')
print(f'data.edge_index --> 4/2 = 2 undirected edges')
print(data.is_directed())
print(data.has_self_loops())

Data(x=[3, 1], edge_index=[2, 4])
data.x = 3 nodes each with 1 feature
data.edge_index --> 4/2 = 2 undirected edges
False
False


In [4]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)

Cora()


In [5]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_idx = data.x, data.edge_index
        
        x = self.conv1(x, edge_idx)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_idx)
        x = F.log_softmax(x, dim=1)

        return x


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)

optim = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [10]:
model.train()

for epoch in range(200):
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    
    optim.zero_grad()
    loss.backward()
    optim.step()

In [11]:
model.eval()

pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()

accuracy = int(correct) / int(data.test_mask.sum())

print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8030


In [12]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(dataset.__dict__)

{'name': 'Cora', 'root': '/tmp/Cora', 'transform': None, 'pre_transform': None, 'pre_filter': None, '_indices': None, 'data': Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708]), 'slices': None, '_data_list': None, 'split': 'public'}


In [21]:
print(dataset.__dir__())
print(dataset.__dict__)
print(dataset[0].__dir__())
print(dataset[0].__dict__)
print(dataset.num_classes)
print(dataset[0].num_nodes)

for i in dataset[0]:
    print(i)


['name', 'root', 'transform', 'pre_transform', 'pre_filter', '_indices', 'data', 'slices', '_data_list', 'split', '__module__', '__doc__', 'url', '__init__', 'raw_dir', 'processed_dir', 'raw_file_names', 'processed_file_names', 'download', 'process', '__repr__', '__parameters__', 'num_classes', 'len', 'get', 'collate', 'copy', 'indices', 'num_node_features', 'num_features', 'num_edge_features', 'raw_paths', 'processed_paths', '_download', '_process', '__len__', '__getitem__', 'index_select', 'shuffle', '__add__', '__orig_bases__', '__dict__', '__weakref__', '__slots__', '__new__', '__class_getitem__', '__init_subclass__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__format__', '__sizeof__', '__dir__', '__class__']
{'name': 'Cora', 'root': '/tmp/Cora', 'transform': None, 'pre_transform': None, 'pre_filter': None, '_indices': None, 'data': Data(x=[